This notebook loads a spreadsheet from an excel, cleans it and converts it into a fixture compatible with the DB schema defined in Django.
  
A cleaning function which accepts a panda's df and return a pandas dataframe is defined for each individual sheet.  
  
To process more sheets. define a new cleaning function and add it to the `cleaning_func_factory` function, following the consistent naming convention. functions are typically named in the format `def clean_df_{FILE_NAME}_{SHEET_NAME}(df):`


In [33]:
import os, sys
import pandas as pd
import numpy as np
import re

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
sys.path.append('../src')

import utils

from fixtures import df_to_json_list, write_fixture_to_json

In [34]:
def load_pandas_excel(fname):
    
    fldr_path = '..','data','raw'
    fpath = os.path.join(*fldr_path, fname)
    xl = pd.ExcelFile(fpath)
    
    return xl

In [35]:
def load_sheet_from_xl(fname, sheet_name):
    """
    load sheet from xl file
    """
    xl = load_pandas_excel(fname)
    df = xl.parse(sheet_name)
        
    return df

In [36]:
def generic_clean_df(df):
    """
    drop NaN rows & duplicates, set_index and convert col names to camel case
    generic dataframe cleaning function for sheets that need no special renaming or index mapping
    """

    df = df.dropna(how='all')
    df = df.drop_duplicates()
    df.columns = [utils.convert_to_camelcase(col) for col in df.columns]
    
    if 'index' in df.columns:
        df['index'] = df['index'].astype(int)
        df = df.set_index('index')

    return df

In [37]:
def clean_df_resources_languages(df):
    
    cols_to_keep = ['index', 'language','index_language', 'equivalentClasses',]   # language hyperlinks example url https://bioportal.bioontology.org/ontologies/SNOMEDCT/?p=classes&conceptid=http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F297301005
    df = df.loc[: ,cols_to_keep]

    #handle NaNs
    df['index_language'] = df['index_language'].astype(pd.Int64Dtype()) # allows column to have NaNs pd.Int64Dtype()
    df['equivalentClasses'] = df['equivalentClasses'].astype(str)
    
    #rename cols # to avoid a naming clashes in django.
    rename_dict = {'language':'name', 
                   'index_language':'parent_language'}
    
    df = df.rename(columns=rename_dict)
    
    df = generic_clean_df(df)

    return df

In [38]:
def clean_df_disorders_disorder_categories(df):
    
    cols_to_keep = ['index', 'disorder_category']   # language hyperlinks example url https://bioportal.bioontology.org/ontologies/SNOMEDCT/?p=classes&conceptid=http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F297301005
    df = df.loc[: ,cols_to_keep]

    df = generic_clean_df(df)
    
    #rename cols # to avoid a naming clashes in django.
    rename_dict = {'disorder_category':'name'}
    df = df.rename(columns=rename_dict)

    return df
    

In [39]:
def get_surname_and_initials_from_string(in_string):
    """use regex to extract inidividual author names & initials from list of authors
    returns: list of tuples, [(surname, first_initial, middle_initial),]
    
    example: in_string='Achenbach, T. M., & Rescorla, L. A.'
            returns [('Achenbach','T','M'), ('Rescorla','L','A')]
    
    """
    
    expression = "([A-Z][a-z]+?[A-Za-z\s]+)([,]?)([\s]?)([A-Za-z]?)([.]?[,\s]?)([A-Z]?[,]?)"
    matches = re.findall(expression, in_string)

    names_lst = []
    for match in matches:
        surname = match[0]
        first_initial =  match[3]
        middle_initial = match[-1]
        
        names_lst.append((surname, first_initial, middle_initial ))
    
    return names_lst

In [40]:

def make_names_df_from_author_lists(ser_authors):
    """
    ser_authors:pd.Series, with lists of authors as a string
    returns: pd.DataFrame with column 'surname','first_initial','middle_initial'
    """

    ser_authors = ser_authors.drop_duplicates().dropna()
    ser_authors = ser_authors.astype(str)

    overall_authors_lst = []

    for auth_lst in ser_authors:
        # add each set of match results to the overall list
        overall_authors_lst += get_surname_and_initials_from_string(auth_lst)

    df = pd.DataFrame(overall_authors_lst, columns = ['surname','first_initial','middle_initial',] )
    df = df.drop_duplicates()
    
    return df


In [41]:
def create_authors_fixture():
    """
    special case function to create an authors fixture
    """
    sheet_name = 'questionnaires'
    input_fname = 'assessments.xlsx'
    app_name = 'assessments'
    model_name = 'Author'
    output_fname = 'Author.json'
    df = load_sheet_from_xl(input_fname, sheet_name)

    ser_authors = df['authors']
    df = make_names_df_from_author_lists(ser_authors)
    df = df.replace('',np.nan)
    
    fname, fixture_lst = df_to_json_list(df,
                                        app_name,
                                        model_name,
                                        file_name_modifier='',
                                        use_df_index_as_pk=False,
                                        create_datetimefield_name=None,
                                        created_by_field_name=None)
    write_fixture_to_json(fixture_lst, output_fname, output_folder='default')
    
    return

In [42]:
def clean_df_disorders_severities(df):

    df = df.set_index('index')
    keep_cols = ['severity', 'definition']
    df = df.loc[:,keep_cols]

    return df

In [43]:
def clean_df_disorders_disorders(df):

    keep_cols = ['S',
                 'index_disorder_category',
                #  'index_disorder_subcategory',
                #  'index_disorder_subsubcategory',
                #  'index_disorder_subsubsubcategory',
                 'disorder',
                #  'equivalentClasses',
                 'ICD9CM',
                 'ICD10CM',
                 'index_diagnostic_specifier',
                #  'index_diagnostic_inclusion_criterion',
                #  'index_diagnostic_inclusion_criterion2',
                #  'index_diagnostic_exclusion_criterion',
                #  'index_diagnostic_exclusion_criterion2',
                 'index_severity',
                 'note']

    df = df.loc[:, keep_cols]
    df.columns = [col.lower() for col in df.columns]
    df['s'] = df['s'].astype(int)
    df = df.rename(columns={'s':'index'})
    
    df = generic_clean_df(df)

    df['index_severity'] = df['index_severity'].astype(pd.Int64Dtype())

    return df

In [44]:
def clean_df_disorders_diagnostic_specifiers(df):

    df['equivalentClasses'] = df['equivalentClasses'].astype(str)
    df = generic_clean_df(df)
        
    return df

In [45]:
def cleaning_func_factory(fname, sheet_name):
    """
    return the appropriate df cleaning function by looking up a dictionary
    the functions should be saved in the dictionary in the format FileName__SheetName
    """
    file_sheet = fname.split('.')[0] + "__" + sheet_name
    
    cleaning_funcs = {}
    cleaning_funcs["resources__languages"] = clean_df_resources_languages
    cleaning_funcs["disorders__disorder_categories"] = clean_df_disorders_disorder_categories
    cleaning_funcs['disorders__severities'] = clean_df_disorders_severities
    cleaning_funcs['disorders__disorders'] = clean_df_disorders_disorders
    cleaning_funcs['disorders__diagnostic_specifiers'] = clean_df_disorders_diagnostic_specifiers
    cleaning_funcs['disorders__diagnostic_criteria'] = generic_clean_df
#     cleaning_funcs[''] = 
    
    return cleaning_funcs[file_sheet]

In [46]:
def process_one_sheet(fname, sheet_name, app_name, model_name):
    
    df = load_sheet_from_xl(fname, sheet_name)
    # look up the cleaning function based on file and sheet name 
    df = cleaning_func_factory(fname, sheet_name)(df)

    fname, fixture_lst = df_to_json_list(df,
                                        app_name,
                                        model_name,
                                        file_name_modifier='',
                                        use_df_index_as_pk=True,
                                        create_datetimefield_name=None,
                                        created_by_field_name=None)

    write_fixture_to_json(fixture_lst, fname, output_folder='default')
    
    return

In [47]:
def get_all_unique_col_vals_from_excel(pandas_xl, col_name):
    """
    iterate through all sheets of a pandas ExcelFile object and return
    a series that concates all columns with the name col_name
    col_name:str, name of column that is in multiple sheets
    pandas_xl:pd.ExcelFile
    
    returns pd.Series
    """
    sheet_names = pandas_xl.sheet_names  # see all sheet names

    ser_lst = []
    col_found_count=0

    for sheet_name in sheet_names:
        df = pandas_xl.parse(sheet_name)
        if col_name in df.columns:
            col_found_count+=1

            ser = df[col_name].dropna(how='any')
            ser_lst.append(ser)

    ser_all = pd.concat(ser_lst).drop_duplicates().dropna(how='any')
    print(f"column '{col_name}' found in {col_found_count} sheets")
    
    
    return ser_all

In [52]:
def split_comma_separated_series(ser, drop_duplicates=True):
    """
    accept a series with entries containing comma separated values.
    split and return series where entries are individual values
    drops duplicated values by default
    
    ser: pd.Series
    """
    
    ser = ser.astype(str).str.split(',')
    
    flat_lst = []  
    for lst in ser:
        flat_lst.extend(lst) 
    
    ser = pd.Series(flat_lst, name=ser.name)
    
    ser = ser.dropna(how='any')
    ser = ser.str.strip(to_strip=' ')

    if drop_duplicates:
        ser = ser.drop_duplicates()

    return ser

In [60]:
def write_all_unique_col_vals_to_fixture(pandas_xl, col_name, app_name, col_case_func=str.lower):
    """
    col_name:str, name of column that is in multiple sheets
    pandas_xl:pd.ExcelFile
    app_name: str, app name in django
    returns pd.Series
    
    """
    model_name = col_name.title() 

    ser = get_all_unique_col_vals_from_excel(pandas_xl, col_name)
    ser = split_comma_separated_series(ser)
    ser = ser.sort_values(ignore_index=True)
    if col_case_func:
        ser.name = col_case_func(ser.name)
    
    df = ser.to_frame()

    output_fname, fixture_lst = df_to_json_list(df,
                                        app_name,
                                        model_name,
                                        file_name_modifier='',
                                        use_df_index_as_pk=False,
                                        create_datetimefield_name=None,
                                        created_by_field_name=None)
    write_fixture_to_json(fixture_lst, output_fname, output_folder='default')
    
    return fixture_lst

#### Temporary Working Space
remove this section before converting to .py script

inputs = {
    'sheet_name' : 'diagnostic_criteria',
    'fname' : 'disorders.xlsx',
    'model_name' : 'DiagnosticCriterion',
    'app_name' : 'disorders',
    }

sheet_name = inputs['sheet_name']
fname = inputs['fname']
model_name = inputs['model_name']
app_name = inputs['app_name']

# MAIN

In [42]:
# special case functions for making tables that did not already have their ownsheet
create_authors_fixture()

wrote fixture to ../data/processed/fixtures/Author.json


In [64]:
# make fixtures for reference columns that appear in mutliple sheets of a file
inputs = [
    {
    'fname' : 'disorders.xlsx',
    'col_names':['ICD9CM','ICD10CM'],
    'app_name' : 'disorders'
    },
    ]

for d in inputs:
    fname = d['fname']
    app_name = d['app_name']
    col_names = d['col_names']

    pandas_xl = load_pandas_excel(fname)

    for col_name in col_names:
        fixture_lst = write_all_unique_col_vals_to_fixture(pandas_xl, col_name, app_name, col_case_func=str.lower)
    


column 'ICD9CM' found in 5 sheets
wrote fixture to ../data/processed/fixtures/Icd9Cm.json
column 'ICD10CM' found in 5 sheets
wrote fixture to ../data/processed/fixtures/Icd10Cm.json


In [39]:
### Main Sheets export  ###

### format example
#     {
#     'sheet_name' : 'languages',   # as defined in the xl sheet
#     'fname' : 'resources.xlsx',   # as defined for the xl file
#     'model_name' : 'Language',    # the Django model class name
#     'app_name' : 'resources',     # the Django app that the the Model class belongs to
#     },

# uncomment the dictionaries to run all sheets

inputs_lst = [
    {
    'sheet_name' : 'languages',
    'fname' : 'resources.xlsx',
    'model_name' : 'Language',
    'app_name' : 'resources',
    },
    {
    'sheet_name' : 'disorder_categories',
    'fname' : 'disorders.xlsx',
    'model_name' : 'DisorderCategory',
    'app_name' : 'disorders',
    },
    {
    'sheet_name' : 'severities',
    'fname' : 'disorders.xlsx',
    'model_name' : 'Severity',
    'app_name' : 'disorders',
    },
    {
    'sheet_name' : 'disorders',
    'fname' : 'disorders.xlsx',
    'model_name' : 'Disorder',
    'app_name' : 'disorders',
    },
    {
    'sheet_name' : 'diagnostic_specifiers',
    'fname' : 'disorders.xlsx',
    'model_name' : 'DiagnosticSpecifier',
    'app_name' : 'disorders',
    },
    {
    'sheet_name' : 'diagnostic_criteria',
    'fname' : 'disorders.xlsx',
    'model_name' : 'DiagnosticCriterion',
    'app_name' : 'disorders',
    },
    
    ]

In [41]:
for d in inputs_lst:
    try: 
        process_one_sheet(d['fname'], d['sheet_name'], d['app_name'], d['model_name'])
    except FileExistsError as e:
        print(e)

wrote fixture to ../data/processed/fixtures/Language.json
wrote fixture to ../data/processed/fixtures/DisorderCategory.json
wrote fixture to ../data/processed/fixtures/Severity.json
wrote fixture to ../data/processed/fixtures/Disorder.json
wrote fixture to ../data/processed/fixtures/DiagnosticSpecifier.json
wrote fixture to ../data/processed/fixtures/DiagnosticCriterion.json
